In [3]:
# primal barrier method from nocedal 19.6
using LinearAlgebra

# an implementation of the primal barrier method.
# this is algorithmic framework 19.5 on page 584.
function barrier(P, ∇P, x, c; μ=1)
    
    # THIS FUNCTION IS INCOMPLETE
    τ = 1 # could be wrong, dont really understand
    ϵ = 0.001
    
    for i in 1:10000
        x = steepest_descent_barrier(P, ∇P, x, μ, c; ϵ=τ)
        if norm(μ*B(x)) < ϵ
            return x, μ
        end
        μ *= 0.9
        τ *= 0.9
        #println("current x and μ: ", x, " ", μ)
    end
    return x, μ
end

# algorithm 3.1 (backtracking line search)
# modified to work with barrier methods
function backtracking_line_search_barrier(f, ∇f, x, μ, c, p)
    α = .5     # initial step length
    rho = 0.75
    ctol = 1e-4

    function stay_feasible(α)
        while c(x + α*p) < 0
            α *= rho
        end
        return α
    end
    α = stay_feasible(α)

    while f(x + α*p, μ) > f(x, μ) + ctol*α*transpose(∇f(x, μ))*p 
        α *= rho
        α = stay_feasible(α)
    end
    return α
end

# steepest descent modified to be used with barrier method
function steepest_descent_barrier(f, ∇f, x, μ, c; ϵ=1e-3, k=1, c1=1e-4)

    # THIS FUNCTION IS INCOMPLETE
    i = 1
    while norm(∇f(x, μ)) > ϵ
        # compute the direction
        B = -I       # steepest descent
        p = B * ∇f(x, μ)
        
        # determine the step size
        α = backtracking_line_search_barrier(f, ∇f, x, μ, c, p)

        x = x + α*p  # the new iterate
        #i % k == 0 && println("iteration ", i, ": step size = ", α, ", x = ", x)
        i += 1
    end
    
    return x
end

# solve the problem
# min (x1 + 0.5)^2 + (x2 - 0.5)^2
# s.t. x1 >= 0
f(x) = (x[1]+0.5)^2 + (x[2]-0.5)^2
c(x) = x[1]
B(x) = -log(c(x))
P(x,μ) = f(x) + μ*B(x)
∇P(x,μ) = [ 2*(x[1]+0.5)-μ/x[1], 2*(x[2]-0.5) ]  # need this for steepest descent

x0 = [1,1]   # start at a feasible point
(x,μ) = barrier(P, ∇P, x0, c; μ=1.0)
println("solution occurs near x = ", x)


solution occurs near x = [0.00010448393268378049, 0.5000000000000041]


In [ ]:
# The function returns a solution of x1 = 0.00010448393268378049 and x2 = 0.5000000000000041. 
# From looking at the function it can be seen that the minimum occurs at (0,0.5) so the result is very accuate.